In [1]:
from fastai import *
from fastai.text import *

In [2]:
path = untar_data(URLs.IMDB,dest='data')

In [3]:
path.ls()

[PosixPath('data/imdb/test'),
 PosixPath('data/imdb/README'),
 PosixPath('data/imdb/unsup'),
 PosixPath('data/imdb/imdb.vocab'),
 PosixPath('data/imdb/tmp_lm'),
 PosixPath('data/imdb/models'),
 PosixPath('data/imdb/train'),
 PosixPath('data/imdb/imdb'),
 PosixPath('data/imdb/tmp_clas')]

In [4]:
bs=64

In [6]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))

In [7]:
data_lm.show_batch()

idx,text
0,"has to control his rage and use it for good instead of evil , and fight the creatures that have been haunting his nightmares ever since that fateful night . \n \n xxmaj truly earns its b - rated rating , but what was cool about it was that it did n't focus on crappy b - rated xxup cgi graphics . xxmaj in fact , 0 % of"
1,"crazed person that xxmaj flynn really tries to emphasise that she was . i miss more psychological diving into xxmaj fish , not to mention the very little time which was spent on xxmaj fish 's post - capture . xxmaj all in all , interesting for those who are into serial - killers , but mostly a let - down ; however , if the director will make another"
2,"on and on ... you can see every "" twist "" and "" turn "" coming a mile away . xxmaj weird soundtrack by some group called "" xxmaj the xxmaj tiger xxmaj lillies "" will drive most people nuts ... but i thought there was one kind of cool track towards the end . xxmaj or maybe i was just excited because i could tell that the movie was"
3,"xxmaj not only did this movie 's script have craters as big as the grand canyon , the movie seemed to jump from one scene to another leaving the viewers thoroughly puzzled . xxmaj the vampire xxmaj lestat played by xxmaj stuart xxmaj townsend was terrible - having a good body does not make you an actor ! xxmaj the end of the queen was too easy and sudden ,"
4,"unsympathetic , but function as a way to poke fun at the xxmaj hollywood system in general , i guess . xxmaj and i 'm probably giving this movie more credit than it actually deserves . xxmaj the pluses are few and far between , but include ( other than xxmaj mitchell and the dance routine ) , colorful , slightly stylized lighting and a bizarre ending that seems to"


In [8]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [9]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.146008,4.021869,0.294974,22:32


In [10]:
learn.save('fit_head')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.919222,3.864480,0.311036,24:42
1,3.836932,3.786588,0.321007,24:41
2,3.781579,3.734300,0.327437,24:40
3,3.709177,3.695040,0.332090,24:42
4,3.653391,3.662404,0.335745,24:39
5,3.594268,3.638423,0.338590,24:41
6,3.545174,3.620611,0.340572,24:43
7,3.485329,3.607924,0.342235,24:42
8,3.474204,3.603026,0.342891,24:41


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [ ]:
learn.save('fine_tuned')

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
data_lm.save('imdb')

In [ ]:
data_lm = load_data(path,'imdb')

In [ ]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs))

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
#learn.load_encoder('fine_tuned_enc')

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.save('clas')

In [13]:
data_clas = TextDataBunch.load(path/'tmp_clas/','.')

In [12]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
#np.load = np_load_old

In [16]:
learn.load("../../models/clas");

RuntimeError: Error(s) in loading state_dict for SequentialRNN:
	size mismatch for 0.module.encoder.weight: copying a param with shape torch.Size([60004, 400]) from checkpoint, the shape in current model is torch.Size([49948, 400]).
	size mismatch for 0.module.encoder_dp.emb.weight: copying a param with shape torch.Size([60004, 400]) from checkpoint, the shape in current model is torch.Size([49948, 400]).